In [2]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AdQt8qgMYj2nIenmQkEk-2kNHoqPDMMQSs1udsL0pD5zce6j1LQ91nIepd8

Successfully saved authorization token.


In [53]:
# Import CAMS aerosol data, and filter data.
aerosol_data = ee.ImageCollection('ECMWF/CAMS/NRT').filter(ee.Filter.date('2020-06-15', '2020-06-19')).select('total_aerosol_optical_depth_at_550nm_surface').filterMetadata('model_initialization_datetime','equals','2020-06-15T00:00:00')
data_size = aerosol_data.size().getInfo()
print('Number of filtered images:', data_size)


Number of filtered images: 32


In [54]:
# definte AOI over north Africa
AOI_NorthAfrica = ee.Geometry.Polygon([[[-61.0, 58.0], [-61.0, -36.0], [53.0, -36.0], [53.0, 58.0]]]);


In [55]:
# load boundary data (plotting global land boundary)
global_boundary = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');


In [56]:
# visualise feature collection (boundary)
# Define an empty image to paint features to.
empty = ee.Image().byte();

# Paint country feature edges to the empty image.
boundaryOutline = empty.paint(**{
  'featureCollection': global_boundary,
  'color': 1,
  'width': 1,
});


In [106]:
visArgs = {
  'min': 0.000096,
  'max': 3.582552,
  'palette': [
    "5E4FA2",
    "3288BD",
    "66C2A5",
    "ABE0A4",
    "E6F598",
    "FFFFBF",
    "FEE08B",
    "FDAE61",
    "F46D43",
    "D53E4F",
    "9E0142",
  ]
};


def addImageVisualise(image):
    return image.visualize(**visArgs)

def addBoundary(image):
    index = image.get('system:time_start')
    return image.blend(boundaryOutline.visualize(**{'palette': '000000'}))

aeosolVisualise = aerosol_data.map(addImageVisualise)
aeosolVisualiseBoudary = aeosolVisualise.map(addBoundary)

videoArgs = {
  'dimensions': 1000,
  'region': AOI_NorthAfrica,
  'framesPerSecond': 1,
  'crs': 'EPSG:3857'
};

# display video in a link
print(aeosolVisualiseBoudary.getVideoThumbURL(videoArgs));

import geemap
import os
saved_gif = os.path.join(os.path.expanduser('~'), 'Downloads/sahara_dust_2020.gif')
geemap.download_ee_video(aeosolVisualiseBoudary, videoArgs, saved_gif)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/37d8e74067cc1693f191eb6f4d6cd189-3cd32db9c3b1bb82935ef112b4bdff61:getPixels
Generating URL...
Please wait ...
The GIF image has been saved to: /Users/rs/Downloads/sahara_dust_2020.gif


In [107]:
from datetime import datetime, timedelta
import numpy as np

initialization_datetime = aerosol_data.reduceColumns(ee.Reducer.toList(), ["model_initialization_datetime"]).get('list').getInfo()
model_forecast_hour = aerosol_data.reduceColumns(ee.Reducer.toList(), ["model_forecast_hour"]).get('list').getInfo()
model_forecast_datetime = []

for i in range(len(initialization_datetime)):
    
    initialization_datetime_i = datetime.strptime(initialization_datetime[i], '%Y-%m-%dT%H:%M:%S')
    model_forecast_datetime_i = initialization_datetime_i + timedelta(hours=int(model_forecast_hour[i]))
    model_forecast_datetime.append(model_forecast_datetime_i)

    
out_gif = os.path.join(os.path.expanduser('~'), 'Downloads/sahara_dust_2020_saved.gif')  
geemap.add_text_to_gif(
    saved_gif,
    out_gif,
    xy=('3%', '2%'),
    text_sequence=model_forecast_datetime,
    font_size=30,
    font_color='#ffffff',
)
os.system('rm -rf %s'%saved_gif)


0